# Evaluation

In [522]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [550]:
import pandas as pd
import numpy as np
from src import preprocess as prep
from src.evaluation import pick_test_segments, generate_test_segments, evaluate_segments, evaluate
from src.Video import Video
from src.Segment import Segment
from src import search
import random
import cv2

## Parameters

In [603]:
NUM_VIDEOS = 200
GRID_SIZE = 2
BINS = [int(180/10), int(256/10)]
# negative value is average; -2 averages two frames, takes every 2nd frame (only skips one) (if frame_id % 2 == 0).
HIST_FRAME_SKIP = 10
REFRESH = True

# vergeet gebruikte params soms dus print ze maar afentoe
def printParams():
    print('Num. Vid {} - Grid {} - Bins {} - Skip {}'.format(NUM_VIDEOS, GRID_SIZE, BINS, HIST_FRAME_SKIP))

## Load training set / generate test set

In [ ]:
printParams()
training_set = prep.load_training_set(range(1, NUM_VIDEOS+1), GRID_SIZE,
                                      BINS, HIST_FRAME_SKIP, force_refresh=REFRESH)

Num. Vid 200 - Grid 2 - Bins [18, 25] - Skip 10


In [592]:
# Set of 100 custom fragments with duration 20sec
test_set, labels = prep.get_test_video_set(NUM_VIDEOS, GRID_SIZE, BINS, n=20)

processing 20/20

In [593]:
# Print statistics
print("TRAINING SET:")
print("Num. videos:    {:d}".format( len(training_set)) )
print("Num. segments:  {:d}".format( np.sum([len(video.segments) for video in training_set])) )
print("Duration:       {:,.1f} s".format( np.sum([np.sum([segment.duration() for segment in video.segments]) for video in training_set])) )
print("Num frames:     {:d}".format( np.sum([np.sum([segment.num_frames() for segment in video.segments]) for video in training_set])) )
print("Num histograms: {:d}".format( np.sum([np.sum([len(segment.histograms) for segment in video.segments]) for video in training_set])) )

print("\nTEST SET:")
print("Size: {:d}".format( len(test_set) ))

TRAINING SET:
Num. videos:    200
Num. segments:  30070
Duration:       97,332.7 s
Num frames:     2640076
Num histograms: 539954

TEST SET:
Size: 20


## Run model on test set

In [601]:
%%time
results = []

for i, test_segment in enumerate(test_set):
    print("\rSearching segment {}/{}".format(i+1, len(test_set), len(test_segment)), end='', flush=True)
    
    results.append(search.knownImageSearch(test_segment, training_set, cv2.HISTCMP_INTERSECT, 5, 
                                    HIST_FRAME_SKIP, [0]))


Searching segment 20/20

## Evaluate performance

In [602]:
movie_results, start_frame_dist = evaluate(results, labels)

fractions = (movie_results[0] / movie_results[2]*100 if movie_results[2] > 0 else 0, \
             movie_results[1] / movie_results[0]*100 if movie_results[0] > 0 else 0)

print("TEST RESULTS\n")

printParams()
print("\nCorrect video: {:d} / {:d} ({:.1f}%)".format(movie_results[0], movie_results[2], fractions[0]))
print("Inside fragment: {:d} / {:d} ({:.1f}%)".format(movie_results[1], movie_results[0], fractions[1]))
print("Average distance to center of segment: {:.0f} +/- {:.0f} frames (approx. {:.1f} sec)".format(
    start_frame_dist[0], start_frame_dist[1], start_frame_dist[0]/30))

TEST RESULTS

Num. Vid 200 - Grid 2 - Bins [18, 25] - Skip 5

Correct video: 20 / 20 (100.0%)
Inside fragment: 13 / 20 (65.0%)
Average distance to center of segment: 275 +/- 257 frames (approx. 9.2 sec)


# Benchmark

In [596]:
#all metrics
for method in [cv2.HISTCMP_CORREL, cv2.HISTCMP_CHISQR, cv2.HISTCMP_INTERSECT,
               cv2.HISTCMP_BHATTACHARYYA, cv2.HISTCMP_CHISQR_ALT, cv2.HISTCMP_KL_DIV]:
    %timeit -n 1 search.knownImageSearch(test_set[0], training_set, method, 5, HIST_FRAME_SKIP)
    
print()
# channels
for channel in [[0], [1], [0,1]]:
    %timeit -n 1 search.knownImageSearch(test_set[0], training_set, cv2.HISTCMP_CHISQR_ALT, 5, HIST_FRAME_SKIP, channel)   

print()

# best n matches per vid
for matches in [2, 5, 10]:
    %timeit -n 1 search.knownImageSearch(test_set[0], training_set, cv2.HISTCMP_CHISQR_ALT, matches, HIST_FRAME_SKIP, [0])   

3.67 s ± 63.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3.72 s ± 66.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3.54 s ± 18 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3.67 s ± 25 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3.72 s ± 12.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
4.29 s ± 118 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

2.3 s ± 29.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.38 s ± 109 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3.72 s ± 45.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

2.09 s ± 18.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.31 s ± 81.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.69 s ± 122 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
